## 웹크롤링

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
interval = 2

url = "https://sgis.kostat.go.kr/view/catchmentArea/main#"

options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36")

# 크롬창을 띄우지 않고 실행(headless)
options.headless = True
# 윈도우창 사이즈 설정
options.add_argument("window-size=2560x1600")

browser = webdriver.Chrome(options=options)
browser.get(url)

# 윈도우창 최대화
browser.maximize_window()

In [2]:
# 팝업창 "취소" 클릭
cancel = browser.find_element_by_class_name("ques02")
cancel.click()

In [3]:
# 웹페이지 정보 가져오기
soup = BeautifulSoup(browser.page_source, "lxml")

In [4]:
# 행정구역 "도" 클릭
provinces = browser.find_element_by_class_name("selct_91")
provinces.click()
time.sleep(interval)

In [5]:
# 서울특별시 선택
seoul = browser.find_element_by_css_selector("#sido > option:nth-child(1)")
seoul.click()
time.sleep(interval)

In [6]:
# 행정구역 "구" 클릭
districts = browser.find_element_by_class_name("selct_92")
districts.click()
time.sleep(interval)

In [7]:
# 웹 크롤링 및 엑셀파일 만들기
import pandas as pd

# 데이터가 필요한 구 추가 및 제거
districts_list = ["도봉구", "노원구", "강북구"]
districts_list.sort()

# 데이터가 필요한 편의시설 추가 및 제거
categories = ["스포츠센터", "응급의료기관", "공원", "사회복지시설"] # 필요한 시설 추가하기

# 엑셀파일 이름 설정 및 생성
writer = pd.ExcelWriter('편의시설 데이터.xlsx', engine="openpyxl") 

for district in districts_list:
    select_district = browser.find_element_by_xpath(f"//option[contains(text(),'{district}')]")
    select_district.click()
    time.sleep(interval)
    print(f'{district} 시작합니다.') # "구" 선택이 잘 되었는지 확인
    
    # 빈 리스트 생성
    result = []  
    for category in categories:
        # 편의시설 분류 창 스크롤을 내리기
        small_window = browser.find_element_by_id("mCSB_1")
        browser.execute_script("arguments[0].scrollBy(0,400)", small_window)

        # 리스트 내 카테고리 선택 및 클릭
        select_category = browser.find_element_by_xpath(f"//a[contains(text(),'{category}')]")
        select_category.click()
        time.sleep(interval)
        soup = BeautifulSoup(browser.page_source, "lxml")
        category_data = soup.select_one("ul.res_li") 
        
#         print(category) # 카테고리 선택이 잘 되었는지 확인
        time.sleep(interval)
        for item in category_data :

            # 편의시설 별 데이터 추출
            name = item.select('li > span[class="sa_txt01"]')[0].get_text()
            category_name = item.select('li > span[class="txt02"]')
            location = item.select('li > div[class="txt03"]')
            
            if category_name and location != None:
                category_name = item.select('li > span[class="txt02"]')[0].get_text()
                location = item.select('li > div[class="txt03"]')[0].get_text()
            else :
                category_name = soup.find("span", attrs={"id":"ftsdText"}).get_text()
                location = '자료가 없습니다.'

            data = [district, name, category_name, location]
            result.append(data)
#             print(name, category_name, location) # 각 시설 별 데이터가 잘 출력되고 있는지 확인


        # 데이터프레임으로 입력 및 엑셀로 추출
        df = pd.DataFrame(result)
        df.columns = ["district", "name", "category_name", "location"]
        df.to_excel(writer, index=False, sheet_name=district)
        
        
#        time.sleep(interval) - 컴퓨터 사양에 따라 시간 늘리기, 기본은 2초

        # 카테고리로 돌아가기 위해 뒤로가기 버튼 클릭
        backToCategory = browser.find_element_by_id("mapLocation_2")
        backToCategory.click()
        time.sleep(interval)
    print(f'{district} 완료합니다.', "\n")

print("웹 크롤링을 종료합니다.")
writer.save()
print("엑셀에 저장을 완료했습니다.")

강북구 시작합니다.
강북구 완료합니다. 

노원구 시작합니다.
노원구 완료합니다. 

도봉구 시작합니다.
도봉구 완료합니다. 

웹 크롤링을 종료합니다.
엑셀에 저장을 완료했습니다.


In [8]:
# 웹드라이버 종료
print("웹드라이버를 종료합니다.")
quit()

웹드라이버를 종료합니다.


## 데이터 전처리

In [27]:
import pandas as pd
districts_list = ["도봉구", "노원구", "강북구"]
districts_list.sort()
categories = ["스포츠센터", "응급의료기관", "공원", "사회복지시설"]

# 전처리된 데이터를 저장하기 위해 엑셀파일 만들기
writer = pd.ExcelWriter('편의시설 데이터(전처리 후).xlsx', engine='openpyxl')

# 전처리를 위한 키워드 추가 및 제거
keywords = ['장애인','육아','청소년','아동','현재 시군구에는 해당 시설 유형이 존재하지 않습니다.']

# 빈 데이터프레임을 생성
df_count = pd.DataFrame(columns=categories)

for district in districts_list:
    df_district = pd.read_excel('편의시설 데이터.xlsx', sheet_name=district) # raw 데이터 파일 불러오기 
    print(f'필터링 전 {district} : {df_district["name"].count()}개')
    
    filterings = df_district[df_district['name'].str.contains('|'.join(keywords))]
    df_district.drop(filterings.index, inplace=True)    
    print(f'필터링 후 {district} : {df_district["name"].count()}개', "\n")
        
    # 필터링 된 데이터를 새로운 액셀파일에 저장
    df_district.to_excel(writer, index=False, sheet_name=district)
    
    
    # 편의시설 개수를 빈 데이터프레임에 추가  
    df_district.drop(columns=['name','location'], inplace=True)
    df_temp = df_district.groupby(by='category_name').count().T
    df_temp.rename(index={'district':district}, inplace=True)
    df_count = df_count.append(df_temp)

# 결측치 처리 후 엑셀파일에 저장
df_count.fillna(0, inplace=True)
print("********** 각 자치구의 편의시설 개수 **********", "\n")
print(df_count)
df_count.to_excel(writer, index=True, sheet_name='total district')
writer.save()

필터링 전 강북구 : 263개
필터링 후 강북구 : 237개 

필터링 전 노원구 : 454개
필터링 후 노원구 : 425개 

필터링 전 도봉구 : 281개
필터링 후 도봉구 : 255개 

********** 각 자치구의 편의시설 개수 ********** 

     스포츠센터  응급의료기관  공원  사회복지시설
강북구    193       0   3      41
노원구    339       6   0      80
도봉구    211       2   2      40
